In [1]:
!pip install -q pyspark

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# Set Spark configuration
conf = SparkConf().setAppName('SparkTextProcessing')
conf.set("spark.hadoop.fs.defaultFS", "file:///")  # Set filesystem to local

# Initialize SparkContext
sc = SparkContext.getOrCreate(conf=conf)

# Initialize SparkSession
sqlContext = SparkSession.builder \
    .master("local[*]") \
    .appName("Colab") \
    .config("spark.driver.memory", "4g") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [3]:
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

st = LancasterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
input_file = sc.textFile("tiny_wikipedia.txt.gz")

In [5]:
#input_file = input_file.sample(True, .0001)

In [6]:
from dateutil.parser import parse
import datetime

def _fix_date(date):
  try:
    bool(parse(date))
    dt = parse(date, default=datetime.datetime(300, 1, 1))
    if dt.year != 300:
        date = dt.strftime("%B %d %Y")
    else:
        date = dt.strftime("%B %d")
    return date
  except:
    return date

In [7]:
import re
def remove_special_chars(word):
  if not word.isalpha():
     word = re.sub(r'[^\w\s]', '', word)
  return word

def identify_dates(expression, doc):
  matches = re.findall(expression, doc)
  dates = [match[0] for match in matches if match[0]]

  doc = doc.split()
  for date in dates:
    doc.append(_fix_date(date))

  return doc

In [8]:
# recognizing dates
month = r"([Jj]anuary|[Ff]ebruary|[Mm]arch|[Aa]pril|[Mm]ay|[Jj]une|[Jj]uly|[Aa]ugust|[Ss]eptember|[Oo]ctober|[Nn]ovember|[Dd]ecember)"
day_and_year = r"\s(\d{1,4})(?:st|nd|rd|th)?,?\s?(\d{4})?"
#day_and_year = r"\s(\d{1,4}),?\s?(\d{4})?"

expression = f'({month}{day_and_year})'

slashed_dates = r"\b(\d{1,2})/(0?\d{1,2}|1[0-2])/(?:\d{2}|\d{4})\b"
slashed_expression = f'({slashed_dates})'

date_expression = expression + r"|" + slashed_expression


In [9]:
stop_words = stopwords.words("english")
stop_words.append('')
stop_words.append('also')

In [10]:
all_stems = input_file.map(lambda line: \
 [word_lower for word in identify_dates(date_expression, line)[1:] \
  if (word_lower := remove_special_chars(word.lower())) not in stop_words])

all_stems_kv = all_stems.map(lambda doc: [(st.stem(word), 1) for word in doc])

In [11]:
global_freq = all_stems_kv.flatMap(lambda word: word)\
.reduceByKey(lambda a, b: a + b)

In [12]:
doc_freq = all_stems_kv.map(lambda words: [pair for pair in set(words)])\
.flatMap(lambda word: word)\
.reduceByKey(lambda a, b: a + b)\
.sortBy(lambda x: x[0])

In [13]:
alphabetized = doc_freq.keys().zipWithIndex()

In [16]:
dictionary_df = alphabetized.map(lambda x: (x[0],)).toDF(["word"])

# Save DataFrame as text file
dictionary_df.write.text("wot.txt")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 1) (Charlie-PC executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.api.python.PairwiseRDD.compute(PythonRDD.scala:128)
	at org.apache.spark.rdd.RDD

In [14]:
dictionary = alphabetized.map(lambda x: f"{x[0]}")
dictionary.saveAsTextFile("wolf.txt")

Py4JJavaError: An error occurred while calling o76.saveAsTextFile.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSy

In [ ]:
unigram = alphabetized.join(doc_freq).join(global_freq)

In [ ]:
unigram = unigram.sortBy(lambda word: -word[1][1])\
.map(lambda word: f"{word[1][0][0]} {word[0]} {word[1][0][1]} {word[1][1]}")

unigram.saveAsTextFile("unigrams.txt")